# Image Stereoscopic Reconstruction (Pipeline)

In this notebook, we explore the process of image translation, in order to obtain a frontal view of an architectural object from the corresponding lateral view, with possible image enhancements (inclusion of new details, inpainting, etc.).
To achieve this, we are going to use an attention based, Chain of Thoughts (CoT) driven generative process, which includes an LLM coupled with a Conditional Latent Diffusion Model (in our example, we are using Qwen 2.5 Image Edit).

## Setup

In [ ]:
%pip install -r requirements.txt

In [ ]:
import base64
import io
import json
import ollama
import chromadb
from matplotlib import image as mpimg
from matplotlib import pyplot as plt

### Utility Functions

In [ ]:
def encode_image(image_path) -> str:
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def decode_and_show_image(base64_img, img_format: str):
    decoded_bytes = io.BytesIO(base64.b64decode(base64_img))
    decoded_image = mpimg.imread(decoded_bytes, format=img_format)
    
    plt.imshow(decoded_image, interpolation='nearest')
    plt.show()

### Ollama

We make use of [Ollama](), a local LLM orchestrator.
Feel free to experiment with other vision models of your taste ([list of available ones](https://ollama.com/search?c=vision)).

In [ ]:
OLLAMA_URL = "http://localhost:11434"   # Feel free to change if your Ollama port is different
MODEL = "qwen2.5vl:32b"                 # Our approach is tested with and works best with Qwen2.5-VL 32B.

%ollama pull $MODEL
%ollama serve

### Vector Store

We make use of [ChromaDB](https://www.trychroma.com/), a lightweight and easy to set up in-memory vector store.
Documentation can be found [here](https://docs.trychroma.com/docs/overview/getting-started).

In [ ]:
chroma_client = chromadb.EphemeralClient()  #By default, we use an in-memory approach which does not persist anything for this demo.
collection = chroma_client.create_collection(name="eustachian_collection")

Adding two images to the vector store, for final image details enrichment.

In [ ]:
collection.add(
    ids=["id1", "id2"],
    documents=[
        f"""
        {{ 
            "caption": "A statue of St. Eustace, patron saint of Matera, suited in armor with a golden plume, holding a spear upright in its right hand.",
            "base64": "{encode_image('assets/stEustace.jpg')}"
        }}
        """,
        f"""
        {{ 
            "caption": "A statue of St. Vitus, suited in light armor and a red cape, bringing a silver cross in is left hand, followed by two dogs of the same breed, of brown and black.",
            "base64": "{encode_image('assets/stVitus.jpg')}"
        }}
        """
    ]
)

#### Example of querying

In [ ]:
result = collection.query(
    query_texts=["This is a query document about a saint followed by dogs"],
    n_results=1
)
print(json.loads(result))

## Phase 1: Prospective Change

To solve this purpose, we make use of Qwen 2.5 Image Edit, an **instructive** T2I model capable of image generation, image editing and in-context image editing, over a CoT-LLM infrastructure.

### Setup

Cloning from public [Qwen Huggingface repo](https://huggingface.co/Qwen/Qwen-Image-Edit)

In [ ]:
!git clone https://huggingface.co/spaces/Qwen/Qwen-Image-Edit

In [ ]:
%pip install git+https://github.com/huggingface/diffusers
%pip install -r Qwen-Image-Edit/requirements.txt

In [ ]:
import os
from PIL import Image
import torch
from diffusers import QwenImageEditPipeline

In [ ]:
pipeline = QwenImageEditPipeline.from_pretrained("Qwen/Qwen-Image-Edit")
print("Pipeline loaded.")

In [ ]:
pipeline.to(torch.bfloat16) # Inference in mixed precision (raccomended for faster inference)
pipeline.to("cuda")         # Change to "cpu" if CUDA is not supported by your machine. The inference will be slower.
pipeline.set_progress_bar_config(disable=False)

In [ ]:
def generate_image(inputs):
    with torch.inference_mode():
        output = pipeline(**inputs)
        output_image = output.images[0]
        output_image.save("output_edit.jpeg")
        print("Image generated (saved!)")
    return output_image

def make_inputs(image: Image, prompt: str, neg_prompt = " "):
    return {
    "image": image,
    "prompt": prompt,
    "generator": torch.manual_seed(0),
    "true_cfg_scale": 4.0,
    "negative_prompt": neg_prompt,
    "num_inference_steps": 50,
    }

### Inference

#### Solid Rotation

In [ ]:
image = Image.open("./.jpeg").convert("RGB")
prompt = "Make the main subject appear in front view."
inputs = make_inputs(image, prompt)
output_image = generate_image(inputs)

In [ ]:
output_image

### Further improvements

These operations aim to edit the previous image in order to have better quality images, but are not strictly necessary.
You can skip these steps if you wish.

#### Colorization

In [ ]:
image = Image.open("./output_edit.jpeg").convert("RGB")
prompt = "Colorize the image."
inputs = make_inputs(image, prompt)
output_image = generate_image(inputs)

In [ ]:
output_image

#### Resolution Upscaling

In [ ]:
image = Image.open("./output_edit.jpeg").convert("RGB")
prompt = "Enhance the resolution of the image."
inputs = make_inputs(image, prompt)
output_image = generate_image(inputs)

In [ ]:
output_image

## Phase 2: Image RAG

## Final Result